In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-insincere-questions-classification/sample_submission.csv
/kaggle/input/quora-insincere-questions-classification/embeddings.zip
/kaggle/input/quora-insincere-questions-classification/train.csv
/kaggle/input/quora-insincere-questions-classification/test.csv


In [2]:
raw_train = pd.read_csv("/kaggle/input/quora-insincere-questions-classification/train.csv")

In [3]:
test_df = pd.read_csv("/kaggle/input/quora-insincere-questions-classification/test.csv")

In [4]:
sub_df = pd.read_csv("/kaggle/input/quora-insincere-questions-classification/sample_submission.csv")

In [5]:
raw_train.head(10)

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
5,00004f9a462a357c33be,"Is Gaza slowly becoming Auschwitz, Dachau or T...",0
6,00005059a06ee19e11ad,Why does Quora automatically ban conservative ...,0
7,0000559f875832745e2e,Is it crazy if I wash or wipe my groceries off...,0
8,00005bd3426b2d0c8305,"Is there such a thing as dressing moderately, ...",0
9,00006e6928c5df60eacb,Is it just me or have you ever been in this ph...,0


In [6]:
#sincere
print(raw_train[raw_train.target == 0].question_text.values[:10])

['How did Quebec nationalists see their province as a nation in the 1960s?'
 'Do you have an adopted dog, how would you encourage people to adopt and not shop?'
 'Why does velocity affect time? Does velocity affect space geometry?'
 'How did Otto von Guericke used the Magdeburg hemispheres?'
 'Can I convert montra helicon D to a mountain bike by just changing the tyres?'
 'Is Gaza slowly becoming Auschwitz, Dachau or Treblinka for Palestinians?'
 'Why does Quora automatically ban conservative opinions when reported, but does not do the same for liberal views?'
 'Is it crazy if I wash or wipe my groceries off? Germs are everywhere.'
 'Is there such a thing as dressing moderately, and if so, how is that different than dressing modestly?'
 'Is it just me or have you ever been in this phase wherein you became ignorant to the people you once loved, completely disregarding their feelings/lives so you get to have something go your way and feel temporarily at ease. How did things change?']


In [7]:
#insincere
print(raw_train[raw_train.target == 1].question_text.values[:10])

['Has the United States become the largest dictatorship in the world?'
 'Which babies are more sweeter to their parents? Dark skin babies or light skin babies?'
 "If blacks support school choice and mandatory sentencing for criminals why don't they vote Republican?"
 'I am gay boy and I love my cousin (boy). He is sexy, but I dont know what to do. He is hot, and I want to see his di**. What should I do?'
 'Which races have the smallest penis?'
 'Why do females find penises ugly?'
 'How do I marry an American woman for a Green Card? How much do they charge?'
 "Why do Europeans say they're the superior race, when in fact it took them over 2,000 years until mid 19th century to surpass China's largest economy?"
 'Did Julius Caesar bring a tyrannosaurus rex on his campaigns to frighten the Celts into submission?'
 "In what manner has Republican backing of 'states rights' been hypocritical and what ways have they actually restricted the ability of states to make their own laws?"]


In [8]:
raw_train.target.value_counts(normalize=True)

target
0    0.93813
1    0.06187
Name: proportion, dtype: float64

Creating a Working sample

In [9]:
sample_df = raw_train.sample(100_000, random_state=42)

In [10]:
sample_df

,qid,question_text,target
443046,56d324bb1e2c29f43b12,What is the most effective classroom managemen...,0
947549,b9ad893dc78c577f8a63,Can I study abroad after 10th class from Bangl...,0
523769,6689ebaeeb65b209a412,How can I make friends as a college junior?,0
949821,ba1e2c4a0fef09671516,How do I download free APK Minecraft: Pocket E...,0
1030397,c9ea2b69bf0d74626f46,"Like Kuvera, is ""Groww"" also a free online inv...",0
...,...,...,...
998930,c3c03a307a29c69971b4,How do I research list of reliable charcoal im...,0
66641,0d119aba95ee6684f506,"What are petroleum products, and what is petro...",0
90024,11a46cd148a104b271cf,What are some services that will let you quick...,0
130113,1973e6e2111a0c93193a,What credit card processors do online marketpl...,0


# **Text Preprocessing**
1. Get single sentence 
2. Tokenization
3. Stop word removal
4. Stemming
5. Bag of words

In [11]:
q0 = raw_train[raw_train.target == 0].question_text.values[1]
q0

'Do you have an adopted dog, how would you encourage people to adopt and not shop?'

In [12]:
q1 = raw_train[raw_train.target == 1].question_text.values[0]
q1

'Has the United States become the largest dictatorship in the world?'

# **Tokenization**


In [13]:
import nltk
from nltk.tokenize import word_tokenize

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
q0_tok = word_tokenize(q0)
q0_tok

['Do',
 'you',
 'have',
 'an',
 'adopted',
 'dog',
 ',',
 'how',
 'would',
 'you',
 'encourage',
 'people',
 'to',
 'adopt',
 'and',
 'not',
 'shop',
 '?']

In [16]:
q1_tok = word_tokenize(q1)
q1_tok

['Has',
 'the',
 'United',
 'States',
 'become',
 'the',
 'largest',
 'dictatorship',
 'in',
 'the',
 'world',
 '?']

# Stop Word Removal

In [17]:
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
english_stopwords = stopwords.words('english')
print(english_stopwords)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [20]:
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in english_stopwords]

In [21]:
q0_stp = remove_stopwords(q0_tok)
q0_stp

['adopted', 'dog', ',', 'would', 'encourage', 'people', 'adopt', 'shop', '?']

In [22]:
q1_stp = remove_stopwords(q1_tok)
q1_stp

['United', 'States', 'become', 'largest', 'dictatorship', 'world', '?']

# Stemming

In [23]:
from nltk.stem.snowball import SnowballStemmer

In [24]:
stemmer = SnowballStemmer(language = 'english')


In [25]:
stemmer.stem('running')

'run'

In [26]:
q0_stm = [stemmer.stem(word) for word in q0_stp]
q0_stm

['adopt', 'dog', ',', 'would', 'encourag', 'peopl', 'adopt', 'shop', '?']

In [27]:
q1_stm = [stemmer.stem(word) for word in q1_stp]
q1_stm

['unit', 'state', 'becom', 'largest', 'dictatorship', 'world', '?']

# Lemmatization


In [31]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()


In [29]:
q0_lem = [lemmatizer.lemmatize(word) for word in q0_stp]
q0_lem

['adopted', 'dog', ',', 'would', 'encourage', 'people', 'adopt', 'shop', '?']

In [30]:
q1_lem = [lemmatizer.lemmatize(word) for word in q1_stp]
q1_lem

['United', 'States', 'become', 'largest', 'dictatorship', 'world', '?']

# **Bag of Words**

BoW model is a simple foundational NLP techniques for representing text data as a numerical vector.

In [83]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()


In [84]:
def tokenize(text):
    return [lemmatizer.lemmatize(word) for word in word_tokenize(text)]

In [85]:
vectorizer = CountVectorizer(lowercase=True, 
                             tokenizer=tokenize,
                             stop_words=english_stopwords,
                             max_features=1000)

In [86]:
%%time
vectorizer.fit(sample_df.question_text)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'m", "'re", "'s", "'ve", 'could', 'doe', 'ha', 'might', 'must', "n't", 'need', 'sha', 'wa', 'wo', 'would'] not in stop_words.
  warnings.warn(


CPU times: user 17.1 s, sys: 25.7 ms, total: 17.2 s
Wall time: 17.2 s


CountVectorizer(max_features=1000,
                stop_words=['a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...],
                tokenizer=<function tokenize at 0x7ee8389b6660>)

In [87]:
len(vectorizer.vocabulary_)

1000

In [88]:
vectorizer.get_feature_names_out()[:100]

array(['!', '$', '%', '&', "'", "''", "'m", "'re", "'s", "'ve", '(', ')',
       ',', '-', '.', '1', '10', '100', '11', '12', '12th', '15', '2',
       '20', '2017', '2018', '3', '30', '4', '5', '6', '7', '8', ':', ';',
       '?', '[', ']', '``', 'able', 'abroad', 'accept', 'access',
       'accomplishment', 'according', 'account', 'act', 'action',
       'activity', 'actor', 'actually', 'add', 'address', 'admission',
       'adult', 'advanced', 'advantage', 'advice', 'affect', 'africa',
       'african', 'age', 'ago', 'air', 'allow', 'allowed', 'almost',
       'alone', 'already', 'also', 'always', 'amazon', 'america',
       'american', 'amount', 'analysis', 'ancient', 'android', 'animal',
       'anime', 'another', 'answer', 'anxiety', 'anyone', 'anything',
       'app', 'apple', 'application', 'apply', 'approach', 'apps', 'area',
       'army', 'around', 'art', 'asian', 'ask', 'asked', 'atheist',
       'attack'], dtype=object)

In [89]:
%%time
inputs = vectorizer.transform(sample_df.question_text)

CPU times: user 16.8 s, sys: 4.81 ms, total: 16.8 s
Wall time: 16.8 s


In [90]:
inputs.shape

(100000, 1000)

In [91]:
%%time
test_inputs = vectorizer.transform(test_df.question_text)

CPU times: user 1min 2s, sys: 61.4 ms, total: 1min 2s
Wall time: 1min 2s


In [92]:
test_inputs.shape

(375806, 1000)

# Text Classification

In [93]:
sample_df

,qid,question_text,target
443046,56d324bb1e2c29f43b12,What is the most effective classroom managemen...,0
947549,b9ad893dc78c577f8a63,Can I study abroad after 10th class from Bangl...,0
523769,6689ebaeeb65b209a412,How can I make friends as a college junior?,0
949821,ba1e2c4a0fef09671516,How do I download free APK Minecraft: Pocket E...,0
1030397,c9ea2b69bf0d74626f46,"Like Kuvera, is ""Groww"" also a free online inv...",0
...,...,...,...
998930,c3c03a307a29c69971b4,How do I research list of reliable charcoal im...,0
66641,0d119aba95ee6684f506,"What are petroleum products, and what is petro...",0
90024,11a46cd148a104b271cf,What are some services that will let you quick...,0
130113,1973e6e2111a0c93193a,What credit card processors do online marketpl...,0


In [94]:
inputs.shape

(100000, 1000)

In [95]:
from sklearn.model_selection import train_test_split

In [96]:
train_inputs, val_inputs, train_targets, val_targets = train_test_split(inputs, sample_df.target, 
                                                                        test_size=0.3, random_state=42)

In [97]:
train_inputs.shape

(70000, 1000)

In [98]:
train_targets.shape

(70000,)

In [99]:
val_inputs.shape

(30000, 1000)

In [100]:
val_targets.shape

(30000,)

### Train Logistic Regression model

In [101]:
from sklearn.linear_model import LogisticRegression

In [102]:
model = LogisticRegression(max_iter=1000, solver='sag')

In [103]:
%%time
model.fit(train_inputs, train_targets)

CPU times: user 17.5 s, sys: 932 µs, total: 17.5 s
Wall time: 17.5 s


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(max_iter=1000, solver='sag')

In [104]:
train_preds = model.predict(train_inputs)

In [105]:
train_preds.shape

(70000,)

In [106]:
pd.Series(train_preds).value_counts(normalize=True)

0    0.974286
1    0.025714
Name: proportion, dtype: float64

In [107]:
from sklearn.metrics import accuracy_score
accuracy_score(train_targets, train_preds)

0.9477428571428571

In [108]:
accuracy_score(train_targets, np.zeros(len(train_targets)))

0.9397714285714286

In [109]:
from sklearn.metrics import f1_score
f1_score(train_targets, train_preds)

0.3919547872340426

In [110]:
f1_score(train_targets, np.zeros(len(train_targets)))

0.0

In [111]:
random_preds = np.random.choice((0, 1), len(train_targets))
f1_score(train_targets, random_preds)

0.10552379007555646

In [112]:
val_preds = model.predict(val_inputs)
accuracy_score(val_targets, val_preds)

0.9455333333333333

In [113]:
f1_score(val_targets, val_preds)

0.37585943468296407

## Make Predictions and Submit to Kaggle

In [114]:
test_inputs.shape

(375806, 1000)

In [115]:
test_preds = model.predict(test_inputs)

In [116]:
sub_df

,qid,prediction
0,0000163e3ea7c7a74cd7,0
1,00002bd4fb5d505b9161,0
2,00007756b4a147d2b0b3,0
3,000086e4b7e1c7146103,0
4,0000c4c3fbe8785a3090,0
...,...,...
375801,ffff7fa746bd6d6197a9,0
375802,ffffa1be31c43046ab6b,0
375803,ffffae173b6ca6bfa563,0
375804,ffffb1f7f1a008620287,0


In [117]:
sub_df.prediction = test_preds
sub_df

,qid,prediction
0,0000163e3ea7c7a74cd7,0
1,00002bd4fb5d505b9161,0
2,00007756b4a147d2b0b3,0
3,000086e4b7e1c7146103,0
4,0000c4c3fbe8785a3090,0
...,...,...
375801,ffff7fa746bd6d6197a9,0
375802,ffffa1be31c43046ab6b,0
375803,ffffae173b6ca6bfa563,0
375804,ffffb1f7f1a008620287,0


In [119]:
sub_df.prediction.value_counts(normalize = True)

prediction
0    0.974553
1    0.025447
Name: proportion, dtype: float64